In [1]:
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


# Tratamento


In [2]:
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector

In [3]:
import re


import locale

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

base_path = f"./data/{canais[0].split('@')[-1]}/"

print(base_path)

./data/BauduccoBrasil/


In [4]:


def trata_inscritos(col: pd.Series) -> pd.Series:
    col = col.split(" inscritos")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )

    return int(col.replace(" ", "").replace(",", "."))


def trata_visualizacoes(col: pd.Series) -> pd.Series:
    # treat the characters U+00a0

    # if its a numeric value return it
    if type(col) == int or type(col) == float:
        return col

    col = col.replace("\xa0", "")

    col = col.split(" visualizações")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    return int(col.replace(" ", "").replace(".", "").replace(",", "."))


def trata_data(value):
    meses = {
        "jan": "01",
        "fev": "02",
        "mar": "03",
        "abr": "04",
        "mai": "05",
        "jun": "06",
        "jul": "07",
        "ago": "08",
        "set": "09",
        "out": "10",
        "nov": "11",
        "dez": "12",
    }
    series_data = re.sub(r"\. de", "", value)
    # print(series_data)
    series_data = re.search(r"(\d{1,2} de [a-zA-Z]+ \d{4})", series_data).group(0)
    # replace month
    for k, v in meses.items():
        series_data = series_data.replace(k, v)

    # print(series_data)
    series_data = pd.to_datetime(series_data, format="%d de %m %Y", errors="coerce")
    return series_data




trata videos

In [5]:

videos = pd.read_csv(base_path + "videos.csv")

display(videos.head())
display(videos.columns)

,title,views,likes,date,comments_amount,channel_name,subscribers
0,Bauducco Cozinha | S'mores com Bauducco® Choco...,179 visualizações,13,7 de ago. de 2023,0,Bauducco,"35,3 mil inscritos"
1,Sanduíche de S'mores com Bauducco® Choco Biscuit,147 visualizações,12,5 de ago. de 2023,0,Bauducco,"35,3 mil inscritos"
2,Bauducco Cozinha | Bauducco® Bolo de Chocolate...,136 visualizações,7,31 de jul. de 2023,0,Bauducco,"35,3 mil inscritos"
3,Bauducco® Cozinha | Gran Gatêau de Chocobiscuit,102 visualizações,7,31 de jul. de 2023,0,Bauducco,"35,3 mil inscritos"
4,Bauducco Cozinha | Triângulo de Bauducco® Cho...,117 visualizações,12,31 de jul. de 2023,0,Bauducco,"35,3 mil inscritos"


Index(['title', 'views', 'likes', 'date', 'comments_amount', 'channel_name',
       'subscribers'],
      dtype='object')

In [6]:


videos.assign(
    subscribers=lambda df_: df_["subscribers"].apply(trata_inscritos),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    date=lambda df_: df_["date"].apply(trata_data),
    category="RECENT",
).to_parquet(base_path + "videos.parquet")


# Trata Top videos

In [7]:





import pandas as pd


top_videos = pd.read_csv(base_path + "top_videos.csv")

display(top_videos.head())

,title,views,likes,date,comments_amount,channel_name
0,Um novo olhar,20 mi de visualizações,565,6 de jul. de 2018,0,Bauducco
1,Cookies Bauducco Tatuagem,16 mi de visualizações,375,15 de ago. de 2017,0,Bauducco
2,Chocolomba® Trufa Bauducco,11 mi de visualizações,43,26 de mar. de 2021,0,Bauducco
3,Natal Bauducco - Bebê Chorão,"9,2 mi de visualizações",246,24 de nov. de 2017,0,Bauducco
4,"Pedacinhos do Natal – Uma carta | 60""","7,9 mi de visualizações",54,26 de nov. de 2018,0,Bauducco


In [8]:

top_videos.assign(
    date=lambda df_: df_["date"].apply(trata_data),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    category="TOP",
).to_parquet(base_path + "top_videos.parquet")


In [9]:


top_videos_silver = pd.read_parquet(base_path + "top_videos.parquet")



In [10]:

# # Trata lifetime


df_lifetime = pd.read_csv(base_path + "lifetime.csv")


df_lifetime.head()



def days_on_youtube(value) -> int:
    import datetime

    today = datetime.date.today()
    return (today - value.date()).days



(
    df_lifetime.assign(
        date_joined=lambda df_: df_["date_joined"].apply(trata_data),
        total_views=lambda df_: df_["total_views"].apply(trata_visualizacoes),
        total_subs=lambda df_: df_["total_subs"].apply(trata_inscritos),
        days_since_joined=lambda df_: df_["date_joined"].apply(days_on_youtube),
    )
).to_csv(base_path + "lifetime_clean.csv", index=False)

In [11]:
# # join


videos = pd.read_parquet(base_path + "videos.parquet")
top_videos = pd.read_parquet(base_path + "top_videos.parquet")


# join the two dataframes

videos = pd.concat([videos, top_videos])


videos_sorted = videos.sort_values("channel_name")

videos_sorted["subscribers"] = videos_sorted["subscribers"].fillna(method="ffill")


videos_sorted = videos_sorted.sort_values(["channel_name", "category", "date"])


videos_sorted.to_parquet(base_path + "videos_sorted.parquet")


pd.read_parquet(base_path + "videos_sorted.parquet").to_csv(
    base_path + "videos_sorted.csv", index=False, sep=';'
)

last 30 days

In [12]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

# print(canais)

base_url = "https://socialblade.com/youtube/channel/"

last_30days = pd.read_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet')


def fix_views(view: pd.Series) -> pd.Series:
    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if "K" in x else x)
    view = view.apply(
        lambda x: float(x.replace("M", "")) * 1000000 if "M" in str(x) else x
    )

    return view


last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]), 
    subs=lambda x: fix_views(x["subs"])
)


last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)